In [1]:
import pandas as pd

In [2]:
!pip install spotipy

In [3]:
nothotsongs=pd.read_csv('not_hot_songs.csv', encoding='latin1')
nothotsongs.head()

,Unnamed: 0,title,artist
0,0,Like a Rolling Stone,Bob Dylan
1,1,(I Can't Get No) Satisfaction',The Rolling Stones
2,2,Imagine,John Lennon
3,3,What's Going On,Marvin Gaye
4,4,Respect,Aretha Franklin


In [4]:
hotsongs=pd.read_csv('hot_songs.csv', encoding='latin1')
hotsongs.head()

,Unnamed: 0,title,artist
0,0,Anti-Hero,Taylor Swift
1,1,Lift Me Up,Rihanna
2,2,Unholy,Sam Smith & Kim Petras
3,3,Bad Habit,Steve Lacy
4,4,As It Was,1


In [5]:
from config import *

In [6]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials



In [7]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,
                                                           client_secret= Client_Secret))

In [8]:
def asking_id(df):
        
    df2 = df.copy()
    song_ids = []
    for i in range(0,len(df2),50):
        print("Chunk: ",i)
        titles = list(df2['title'][i:i+50].values)
        artists = list(df2['artist'][i:i+50].values)
        elems = list(zip(titles,artists))
        for item in elems:
            string = "track:" + item[0] + " artist:" + item[1]
            print("Searching for song: ",item[0],"from artist: ", item[1])
            try:
                results = sp.search(string, limit=1)
                song_id = results['tracks']['items'][0]['id']
                song_ids.append(song_id)
            except:
                song_id = "None"
                song_ids.append(song_id)
                print("Song not found!")
        print()
        print("Waiting 10 seconds before the next chunk")
        sleep(10)

    df2['id'] = song_ids
    
    return df2
        
        

In [9]:
from time import sleep

In [10]:
def asking_features(df):
    
    df1=pd.DataFrame()
    
    df3 = df.copy()
    
    for i in range(0,len(df3),50):
        print("Chunk: ",i)
        elems = list(df3['id'][i:i+50].values) #this is my list of ids 
        for item in elems:
            if item != 'None':
                features = sp.audio_features((item))[0]
                my_dict_new = { key : [features[key]] for key in list(features.keys()) }
                new_df=pd.DataFrame(my_dict_new)
                
                df1 = pd.concat([df1, new_df], axis=0) 
          
            else:
        
                print('uri is None')
        print("Waiting 10 seconds before the next chunk")
        sleep(10)

  
    return df3, df1

In [11]:
result  = asking_id(hotsongs)
result = result.loc[~result['id'].str.contains('None')]
df_titles, df_features_hot_song=asking_features(result)

Chunk:  0
Searching for song:  Anti-Hero from artist:  Taylor Swift
Song not found!
Searching for song:  Lift Me Up from artist:  Rihanna
Song not found!
Searching for song:  Unholy from artist:  Sam Smith & Kim Petras
Song not found!
Searching for song:  Bad Habit from artist:  Steve Lacy
Song not found!
Searching for song:  As It Was from artist:  1
Song not found!
Searching for song:  Lavender Haze from artist:  Taylor Swift
Song not found!
Searching for song:  Midnight Rain from artist:  Taylor Swift
Song not found!
Searching for song:  I Like You (A Happier Song) from artist:  Post Malone Featuring Doja Cat
Song not found!
Searching for song:  Bejeweled from artist:  Taylor Swift
Song not found!
Searching for song:  Super Freaky Girl from artist:  Nicki Minaj
Song not found!
Searching for song:  Shirt from artist:  SZA
Song not found!
Searching for song:  Maroon from artist:  Taylor Swift
Song not found!
Searching for song:  I Ain't Worried from artist:  OneRepublic
Song not found

In [12]:
def add_audio_features(df, audio_features_df):
    
    df4=df.copy()
    df5=audio_features_df.copy()
    df4=df4.drop(columns = ['Unnamed: 0',"id"], axis=1).reset_index(drop=True)
    df5=df5.reset_index(drop=True)
    df_final = pd.concat([df4, df5], axis=1)
    return df_final

In [13]:
df_hot_songs_final=add_audio_features(df_titles, df_features_hot_song)
df_hot_songs_final

,title,artist


In [ ]:
result  = asking_id(nothotsongs)
result = result.loc[~result['id'].str.contains('None')]

In [ ]:
df_titles_not_hot, df_features_not_hot_song=asking_features(result)
